# Import Commands

In [3]:
#%qtconsole
#%matplotlib inline

import matplotlib.pyplot as plt
from functools import partial
import numpy as np
import time 
import qcodes as qc
import os
import openpyxl
from qcodes.math_utils import FieldVector
from qcodes import Station, load_or_create_experiment, initialise_database, Measurement, load_by_run_spec, load_by_guid, initialise_or_create_database_at 
from qcodes.utils.dataset.doNd import do1d, do2d, plot

from qcodes_contrib_drivers.drivers.Tektronix.Keithley_2700 import Keithley_2700
from qcodes.instrument_drivers.tektronix.Keithley_2600_channels import Keithley_2600
from qcodes_contrib_drivers.drivers.Oxford.IPS120 import OxfordInstruments_IPS120
from qcodes.instrument.visa import VisaInstrument
from qcodes_contrib_drivers.drivers.StanfordResearchSystems.SIM928 import SIM928
from qcodes.instrument_drivers.stanford_research.SR830 import SR830
from qcodes.instrument_drivers.stanford_research.SR860 import SR860
from qcodes.dataset.plotting import plot_dataset
from qcodes.instrument_drivers.oxford.MercuryiPS_VISA import MercuryiPS
import IPython.lib.backgroundjobs as bg
from plottr.apps import inspectr

# Initialize Instruments

In [4]:
#Magnet Controller
#mag120 = OxfordInstruments_IPS120('ips','TCPIP0::169.254.255.2::7020::SOCKET',23)
#mag120 = OxfordInstruments_IPS120('ips','GPIB0::23::INSTR')
mag120 = MercuryiPS('mips', 'TCPIP0::169.254.255.2::7020::SOCKET')
#Lockins(SR860 and SR830)
rxy1=SR830('Dilbert','GPIB1::6::INSTR' )
rxy2=SR860('Laughlin', 'GPIB1::4::INSTR')
rxy3=SR860('Kiwi','GPIB1::9::INSTR' )
rxy4=SR860('Pangolin', 'GPIB1::12::INSTR')
rxy5=SR830('Wally','GPIB1::1::INSTR' )

Connected to: OXFORD INSTRUMENTS MERCURY IPS (serial:190250049, firmware:2.5.09.000) in 0.15s
Connected to: Stanford_Research_Systems SR830 (serial:s/n48257, firmware:ver1.07) in 0.14s
Connected to: Stanford_Research_Systems SR860 (serial:004479, firmware:V1.51) in 0.05s


C:\Anaconda3\envs\MattMann\lib\site-packages\qcodes\instrument\instrument_base.py:147: QCoDeSDeprecationWarning: Parameter ch1_databuffer on instrument Gilbert does not correctly pass kwargs to its baseclass. A Parameter class must take `**kwargs` and forward them to its baseclass.
  QCoDeSDeprecationWarning,
C:\Anaconda3\envs\MattMann\lib\site-packages\qcodes\instrument\instrument_base.py:147: QCoDeSDeprecationWarning: Parameter ch2_databuffer on instrument Gilbert does not correctly pass kwargs to its baseclass. A Parameter class must take `**kwargs` and forward them to its baseclass.
  QCoDeSDeprecationWarning,


Connected to: Stanford_Research_Systems SR860 (serial:003172, firmware:V1.47) in 0.05s
Connected to: Stanford_Research_Systems SR860 (serial:004478, firmware:V1.51) in 0.05s
Connected to: Stanford_Research_Systems SR830 (serial:s/n48240, firmware:ver1.07) in 0.09s


C:\Anaconda3\envs\MattMann\lib\site-packages\qcodes\instrument\instrument_base.py:147: QCoDeSDeprecationWarning: Parameter ch1_databuffer on instrument Wally does not correctly pass kwargs to its baseclass. A Parameter class must take `**kwargs` and forward them to its baseclass.
  QCoDeSDeprecationWarning,
C:\Anaconda3\envs\MattMann\lib\site-packages\qcodes\instrument\instrument_base.py:147: QCoDeSDeprecationWarning: Parameter ch2_databuffer on instrument Wally does not correctly pass kwargs to its baseclass. A Parameter class must take `**kwargs` and forward them to its baseclass.
  QCoDeSDeprecationWarning,


# Global Parameters for Instruments

In [5]:
excelPath = "C:\\Users\\Admin\\Desktop\\MattMann\\8-3-22.1MM11B4\\MicroOhmicResistance\\MicroOhmicResistance_27,28,13,40,&43_SweepUp.xlsx"
csvPath = "C:\\Users\\Admin\\Desktop\\MattMann\\8-3-22.1MM11B4\\MicroOhmicResistance\\MicroOhmicResistance_27,28,13,40,&43_SweepUp.csv"
dataBasePath = 'C:\\Users\\Admin\Desktop\\MattMann\\8-3-22.1MM11B4\\MicroOhmicResistance\\measurements.db'
sample = "8-3-22.1MM11B4"
exp_name = "MicroOhmicResistance vs Mag Field"
meas_name = 'MicroOhmicResistance_vs_Mag Field'
amp = 0.000000001
#Sweep rate in T/s
#IMPORTANT: Never go above 50mT/min(0.83mT/s)
sweepRate = 0.00083
fieldTarget = 7.5

# Set Up Database for Data Collection

In [6]:
db_file_path = os.path.join(os.getcwd(), dataBasePath)
initialise_or_create_database_at(db_file_path)

# Set Magnetic Field Sweep Parameters

In [7]:
#Prints old sweep rate, sets new one, prints new sweep rate
field = mag120._get_ramp_rate()
print(field)
rate = FieldVector(0.0,0.0,sweepRate)
print(rate)
mag120._set_ramp_rate(rate)

FieldVector(x=0.0, y=0.0, z=0.0008299999999999999)
FieldVector(x=0.0, y=0.0, z=0.00017)


In [8]:
#Prints old target field, sets new one, prints new target field
target = FieldVector(0.0,0.0,fieldTarget)
print(mag120._get_target_field())
mag120._set_target_field(target)
print(mag120._get_target_field())

FieldVector(x=0.0, y=0.0, z=7.5)
FieldVector(x=0.0, y=0.0, z=0.0)


# Set Up Live Plot

In [9]:
jobs = bg.BackgroundJobManager()
jobs.new(inspectr.main, db_file_path)

<BackgroundJob #0: <function main at 0x000000001DC64AF8>>

# Setup Station and Measurement Parameters

In [12]:
station = qc.Station()
station.add_component(mag120)
station.add_component(rxy1)
station.add_component(rxy2)
station.add_component(rxy3)
station.add_component(rxy4)
station.add_component(rxy5)

cutoff_freq_exp = load_or_create_experiment(
    experiment_name=exp_name,
    sample_name=sample
)
context_meas = Measurement(exp=cutoff_freq_exp, station=station, name=meas_name)
context_meas.write_period = 0.1

context_meas.register_parameter(mag120.field_measured)
context_meas.register_parameter(rxy1.X, setpoints=(mag120.field_measured,))
context_meas.register_parameter(rxy2.X, setpoints=(mag120.field_measured,))
context_meas.register_parameter(rxy3.X, setpoints=(mag120.field_measured,))
context_meas.register_parameter(rxy4.X, setpoints=(mag120.field_measured,))
context_meas.register_parameter(rxy5.X, setpoints=(mag120.field_measured,))

# Sweep Magnetic Field and Measure Results for MicroOhmics

In [17]:
workbook = openpyxl.Workbook()

sheet = workbook.active 

c1 = sheet.cell(row = 1, column = 1) 
c1.value = "Mag Field"
c2 = sheet.cell(row = 1, column = 2) 
c2.value = "Resistance(GPIB::6)"
c3 = sheet.cell(row = 1, column = 3) 
c3.value = "Resistance(GPIB::4)"
c4 = sheet.cell(row = 1, column = 4) 
c4.value = "Resistance(GPIB::9)"
c5 = sheet.cell(row = 1, column = 5) 
c5.value = "Resistance(GPIB::12)"
c6 = sheet.cell(row = 1, column = 6) 
c6.value = "Resistance(GPIB::1)"
curr = 2

mag120.ramp('simul')
with context_meas.run() as datasaver:
    while(mag120._get_field().z>0.001):
        set_m = mag120.field_measured.get().z
        get_x1 = rxy1.X.get()/amp
        get_x2 = rxy2.X.get()/amp
        get_x3 = rxy3.X.get()/amp
        get_x4 = rxy4.X.get()/amp
        get_x5 = rxy5.X.get()/amp

        datasaver.add_result((mag120.field_measured, set_m),
                             (rxy1.X, get_x1),
                             (rxy2.X, get_x2),
                             (rxy3.X, get_x3),
                             (rxy4.X, get_x4),
                             (rxy5.X, get_x5))
        
        temp1 = sheet.cell(row = curr, column = 1)
        temp2 = sheet.cell(row = curr, column = 2)
        temp3 = sheet.cell(row = curr, column = 3)
        temp4 = sheet.cell(row = curr, column = 4)
        temp5 = sheet.cell(row = curr, column = 5)
        temp6 = sheet.cell(row = curr, column = 6)
        time.sleep(1)
        (X1, P1)= rxy1.snap('x','p')
        print((X1, P1))
        (X2, P2) = rxy2.get_values('X','P')
        print((X2, P2))
        (X3, P3) = rxy3.get_values('X','P')
        print((X3, P3))
        (X4, P4) = rxy4.get_values('X','P')
        print((X4, P4))
        (X5, P5)= rxy5.snap('x','p')
        print((X5, P5))
        print("\n")
        temp1.value = mag120._get_field().z
        temp2.value = X1/amp
        temp3.value = X2/amp
        temp4.value = X3/amp
        temp5.value = X4/amp
        temp6.value = X5/amp
        curr = curr + 1
        workbook.save(filename=excelPath)
    dataset = datasaver.dataset

Starting experimental run with id: 4. 
(0.000302048, 0.912)
(0.00028951751301, -0.38562053442)
(0.00029265700141, -2.5071179867)
(0.00029387068935, -1.8062912226)
(0.000297965, -3.132)


(0.000301959, 0.51)
(0.00029594750958, -1.6168525219)
(0.00029455433832, -2.4769356251)
(0.00027645347291, -2.7989225388)
(0.000297518, -3.414)


(0.000300856, 0.882)
(0.00029733875999, 0.20377355814)
(0.00029514633934, -1.6393631697)
(0.00029563737917, -2.6685888767)
(0.000297042, -3.3)


(0.000302197, 1.038)
(0.00028656187351, -0.49578973651)
(0.00029611639911, -2.9127066135)
(0.00027890596539, -3.5790565014)
(0.00029874, -3.45)


(0.000301393, 0.642)
(0.00029696896672, 0.038658868521)
(0.00028075592127, -0.86851143837)
(0.00028763533919, -0.79749625921)
(0.00029722, -3.408)


(0.000302525, 0.696)
(0.00028752768412, -1.2716497183)
(0.00028731595376, -1.5054709911)
(0.00027801294345, 0.38310360909)
(0.000297787, -3.486)


(0.000301184, 0.564)
(0.00029618639383, -0.46453312039)
(0.00029752668343, -1.18

(0.000300737, 1.038)
(0.00028913281858, -1.5716366768)
(0.00029309731326, -1.8613886833)
(0.00027931004297, -4.0655369759)
(0.000298949, -3.408)


(0.000302555, 0.666)
(0.00029532576445, 0.082723833621)
(0.00029496868956, -0.22469101846)
(0.00028668317827, 0.010352858342)
(0.000296744, -3.414)


(0.000300677, 0.948)
(0.00029175251257, -1.7612308264)
(0.0002973320079, -0.74866896868)
(0.00028612848837, -3.5352528095)
(0.000298681, -3.354)


(0.000302525, 0.642)
(0.00029519182863, 0.32621663809)
(0.0002930482151, -1.1805315018)
(0.0002898221428, 0.020234433934)
(0.000296773, -3.336)


(0.000300707, 0.96)
(0.00029179139528, -1.1606026888)
(0.00027627783129, -3.1269285679)
(0.00027980117011, -4.0511627197)
(0.00029883, -3.282)


(0.000302376, 0.624)
(0.00029269550578, -1.275116086)
(0.00029998022364, -1.043690443)
(0.00028179041692, -4.2104496956)
(0.000299336, -3.264)


(0.000300409, 0.702)
(0.00029008300044, -1.0140149593)
(0.0003002891026, 0.52836632729)
(0.00028879585443, -3.4576141834

(0.000301423, 1.014)
(0.00028623270919, -0.76984083652)
(0.00029180158162, -3.7689049244)
(0.00028583122185, -1.7273905277)
(0.000297161, -3.234)


(0.00030178, 0.558)
(0.00029684050241, -0.70346164703)
(0.00029856478795, -2.9386751652)
(0.00029244358302, -3.9162201881)
(0.000299187, -3.168)


(0.000301721, 1.062)
(0.00028675043723, -0.14066945016)
(0.00028812649543, -2.1706800461)
(0.0002878982923, -0.31526616216)
(0.000297161, -3.156)


(0.000302227, 0.612)
(0.00029364851071, -0.36892753839)
(0.00030912729562, -1.7889903784)
(0.00029874881147, -0.89410591125)
(0.00029722, -3.216)


(0.000300499, 0.99)
(0.00029152460047, -1.4199706316)
(0.00028945290251, -2.9106116295)
(0.00029146700399, -4.5453267097)
(0.00029877, -3.126)


(0.000302525, 0.678)
(0.00029322912451, 0.66687232256)
(0.00028361839941, -1.8018056154)
(0.0002865777642, -0.19483627379)
(0.000296922, -3.366)


(0.000300827, 1.044)
(0.00029085710412, -0.97310894728)
(0.00027795723872, -2.206461668)
(0.00028307258617, -4.421941

(0.000301274, 1.11)
(0.00028793149977, 0.3484608531)
(0.00028463380295, -1.1128256321)
(0.00028969868436, -3.7015798092)
(0.000298264, -3.528)


(0.000301572, 0.594)
(0.00029404391535, -1.4146039486)
(0.00029020383954, -1.3566838503)
(0.00028476241278, 0.23525050282)
(0.000298055, -3.108)


(0.000302108, 0.726)
(0.00029299035668, 0.42925515771)
(0.00027071079239, -3.542996645)
(0.00028818342253, -0.78450357914)
(0.000297518, -3.138)


(0.000301065, 0.552)
(0.00028969076811, -1.369666338)
(0.00028891194961, -0.34363365173)
(0.00029914543848, -1.3798860312)
(0.000299128, -3.396)


(0.000302168, 1.038)
(0.00029543688288, 0.0040537170134)
(0.0002914465731, -2.5341570377)
(0.00027335539926, -2.2429997921)
(0.000297131, -3.192)


(0.000301631, 0.534)
(0.00029339207686, -1.2315939665)
(0.00028505365481, 0.25601938367)
(0.00028269525501, 0.15750074387)
(0.000297429, -3.138)


(0.000301303, 1.044)
(0.00028938468313, 0.57526415586)
(0.00028611952439, -1.0836316347)
(0.00028234373895, -3.10937356

(0.000301244, 1.086)
(0.0002925216977, -1.0190504789)
(0.00028128540725, -3.6830000877)
(0.00028428551741, -3.2041270733)
(0.000296714, -3.408)


(0.000302823, 0.666)
(0.0002936689707, -1.0664144754)
(0.00028755972744, 0.21724782884)
(0.00028951265267, -0.20280367136)
(0.000298442, -3.198)


(0.000300648, 0.96)
(0.00028995558387, -0.061225809157)
(0.00028869239031, -3.5548143387)
(0.00027816309012, -2.7443494797)
(0.00029725, -3.282)


(0.000302615, 0.636)
(0.00029078012449, -0.43656688929)
(0.0002868315496, 1.4698600769)
(0.00027977579157, -2.5746567249)
(0.000296982, -3.168)


(0.000300409, 0.852)
(0.00028705320437, -0.792144835)
(0.00028991032741, -1.4705396891)
(0.0002987908083, -2.5215845108)
(0.000298532, -3.492)


(0.000303032, 0.684)
(0.00029480221565, -1.4201841354)
(0.00028526957612, -0.79236000776)
(0.00028955232119, -0.25639677048)
(0.000299009, -3.126)


(0.00030026, 0.882)
(0.00028851063689, -0.53091740608)
(0.00029713378171, -0.92196285725)
(0.00029876732151, -3.14413905

...

(5.29888e-05, 0.954)
(4.9761256378e-05, 1.6217190027)
(4.8091522331e-05, -0.025391226634)
(4.0865208575e-05, 3.8675193787)
(5.42703e-05, -1.5)


(5.36444e-05, 0.282)
(4.9592468713e-05, 2.5886142254)
(5.1557690313e-05, 7.5425610542)
(5.8659494243e-05, -6.117641449)
(5.33166e-05, -2.622)


(5.41511e-05, 0.282)
(5.0030164857e-05, -3.1855762005)
(5.1336530305e-05, -7.7777509689)
(5.2778061217e-05, -9.7577867508)
(5.40318e-05, -3.336)


(5.45683e-05, 1.536)
(5.3968218708e-05, -1.2405006886)
(6.0114540247e-05, -3.4771945477)
(4.7971698223e-05, -10.633879662)
(5.39126e-05, -1.974)


(5.39424e-05, 1.644)
(5.4800711951e-05, 1.0177254677)
(5.7474026107e-05, -2.3974032402)
(5.3724819736e-05, -9.7990837097)
(5.40914e-05, -2.166)


(5.43299e-05, 0.156)
(5.0362599723e-05, 3.8314301968)
(5.7437933719e-05, 4.990170002)
(5.9296253312e-05, -6.4207859039)
(5.42405e-05, -2.466)


(5.43895e-05, 0.264)
(4.9275284255e-05, -0.28806835413)
(4.5850745664e-05, 0.082744322717)
(4.5389500883e-05, 1.5280174017)
(5.

(5.39424e-05, 0.48)
(4.9255671911e-05, -2.169355154)
(4.7417750466e-05, -6.9584870338)
(4.3216328777e-05, 2.919342041)
(5.47769e-05, -1.458)


(5.30782e-05, 0.576)
(5.1739443734e-05, -2.332955122)
(5.3701274737e-05, 7.1673870087)
(4.4722783059e-05, 7.0196943283)
(5.34954e-05, -1.98)


(5.33762e-05, 1.5)
(5.3536186897e-05, 0.9742154479)
(5.0157956139e-05, -25.151895523)
(5.7110213675e-05, 5.3612136841)
(5.44193e-05, -2.88)


(5.42703e-05, 0.666)
(5.285287989e-05, 3.3495790958)
(4.427328895e-05, -2.1391913891)
(5.6396966102e-05, -8.6187963486)
(5.57008e-05, -2.274)


(5.31378e-05, 1.65)
(4.9313770432e-05, -1.7806608677)
(5.239449456e-05, 7.2074189186)
(4.3032912799e-05, -3.0403323174)
(5.42405e-05, -3.228)


(5.24523e-05, 0.69)
(5.285115185e-05, -1.7151372433)
(4.7582067054e-05, 7.5035448074)
(5.5202923249e-05, -7.6692447662)
(5.49557e-05, -1.746)


(5.27503e-05, 0.216)
(4.8486625019e-05, 3.9967124462)
(5.8053865359e-05, 3.0817079544)
(5.7522334828e-05, -5.8169288635)
(5.28099e-05, -1.63

(5.2959e-05, 1.368)
(4.8644851631e-05, -2.5382621288)
(4.5073353249e-05, 3.5087962151)
(5.9706839238e-05, 1.6520929337)
(5.25715e-05, -1.68)


(5.3257e-05, 1.02)
(4.8620600865e-05, 3.6798241138)
(4.9003927415e-05, -7.7786374092)
(5.5408610933e-05, 6.8740162849)
(5.34358e-05, -1.116)


(5.29888e-05, 1.584)
(4.7831028496e-05, 3.1777145863)
(4.9051388487e-05, -7.2560648918)
(5.994744788e-05, -0.54777771235)
(5.47471e-05, -1.998)


(5.26609e-05, 1.578)
(4.7359411838e-05, 2.7826206684)
(5.0818718591e-05, -6.9265880585)
(5.9622612753e-05, 3.008739233)
(5.42405e-05, -1.818)


(5.21543e-05, 1.722)
(4.7522909881e-05, 2.3830769062)
(4.7957011702e-05, -7.6287813187)
(5.4229760281e-05, -9.4007301331)
(5.30186e-05, -3.258)


(5.22735e-05, 1.752)
(4.9988248065e-05, 3.1157767773)
(4.7299479775e-05, 7.0469174385)
(5.2206301916e-05, 8.429848671)
(5.43299e-05, -2.58)


(5.21543e-05, 1.722)
(5.2443127061e-05, -2.6797630787)
(4.3323085265e-05, -4.6021351814)
(5.5475691624e-05, 8.2510728836)
(5.25715e-05, 

(5.10218e-05, 0.636)
(5.1420625823e-05, 0.5097848177)
(4.6162644139e-05, 0.88323044777)
(6.0090791521e-05, 2.9637863636)
(5.36146e-05, -1.848)


(5.23629e-05, 1.278)
(4.6223358368e-05, -3.5577464104)
(4.853421342e-05, 9.7936286926)
(5.9427304222e-05, -2.2867045403)
(5.29292e-05, -1.038)


(5.19159e-05, 0.402)
(5.1207312936e-05, 1.6777266264)
(4.3339281547e-05, 1.8621400595)
(4.3990483391e-05, -8.1541719437)
(5.18265e-05, -2.556)


(5.19457e-05, 1.728)
(4.6175147872e-05, 1.5280582905)
(5.3870640841e-05, -6.4337000847)
(5.4327174439e-05, 8.156580925)
(5.3555e-05, -1.578)


(5.18265e-05, 1.692)
(4.8734924349e-05, -2.1790471077)
(4.6001394367e-05, 5.3262429237)
(5.4221989558e-05, -7.7940602303)
(5.3853e-05, -2.04)


(5.20053e-05, 0.438)
(5.1418563089e-05, -0.83332568407)
(5.5612305005e-05, 1.437307477)
(5.2973879065e-05, 6.8871202469)
(5.36742e-05, -2.85)


(5.21543e-05, 1.464)
(5.1054776122e-05, 2.9461951256)
(5.3476400353e-05, 6.750210762)
(4.4332096877e-05, 6.6890716553)
(5.3853e-05, -1

(5.11708e-05, 0.84)
(4.8613816034e-05, 1.2688527107)
(5.1739996707e-05, 0.74524533749)
(5.0018446927e-05, -4.0095105171)
(5.29292e-05, -2.268)


(5.06344e-05, 0.828)
(4.9863367167e-05, 2.9452645779)
(5.776681428e-05, -3.5524852276)
(4.1897259507e-05, 4.0911874771)
(5.19457e-05, -2.97)


(5.06642e-05, 0.888)
(5.1661652833e-05, -0.76922100782)
(4.4718941353e-05, 4.3460969925)
(5.3299616411e-05, -9.0629148483)
(5.29888e-05, -3.264)


(5.11112e-05, 1.794)
(4.7739202273e-05, 3.3705873489)
(4.6426863264e-05, 6.8179216385)
(4.2531224608e-05, -0.19893094897)
(5.36742e-05, -1.662)


(5.13198e-05, 0.15)
(4.613404235e-05, -0.57488161325)
(4.6952609409e-05, 6.836725235)
(4.9271340686e-05, 9.8459014893)
(5.17073e-05, -2.52)


(5.0992e-05, 1.758)
(4.6699849918e-05, 3.5396156311)
(4.2993375246e-05, 2.3314201832)
(4.0925882786e-05, -1.7628887892)
(5.33464e-05, -1.29)


(5.06344e-05, 1.206)
(4.9336078519e-05, 3.2155675888)
(5.5462671298e-05, 0.9209587574)
(5.2733121265e-05, -8.2489595413)
(5.34954e-05,

# Converts Dataset Results to CSV

In [18]:
df = dataset.to_pandas_dataframe()
df.to_csv(csvPath)